In [1]:
# use the raw data :')
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# some hyperparameter type things :)
sr = 200
t = 200
ch = 19
affix = '_raw'

In [2]:
# load from .pkl files ("raw")
raw_features = {}
raw_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    raw_features[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_features{affix}.npy')
    raw_labels[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_labels{affix}.npy').reshape(-1)
    fileList.append((f'subj_{letter}', raw_features[f'subj_{letter}'], raw_labels[f'subj_{letter}']))

In [3]:
# ConvNet_Finger

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_Finger(nn.Module):
    def __init__(self):
        super(ConvNet_Finger, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x

In [4]:
# final NeuralNet
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        nn.Sequential(
            nn.Linear(5, 16),
            nn.Linear(16, 32),
            nn.Linear(32, 5)
        )
        return x

In [13]:
# help :P

def predict(y_train):
    data = y_train
    pred = []
    with torch.no_grad():
        model.eval()
        data = data.to(device)
        predictions = model(data)
        _, predictions = predictions.max(1)
    return predictions

def evaluate(predictions, targets):
    n_samples, n_correct = 0, 0
    # print(targets.shape, predictions.shape)
    # print(type(targets), targets.shape, type(predictions), len(predictions))
    n_samples += targets.size(0)
    n_correct += (predictions == targets).sum().item()
    acc = n_correct / n_samples
    print(f'accuracy = {acc}')
    return acc

def binary_confusion(y_pred, target):
    # matrix represents [pred 0 targ 0, pred 0 targ 1] [pred 1 targ 0, pred 1 targ 1]
    matrix = [[0, 0],[0, 0]]
    for i in range(y_pred.shape[0]):
        #zero stands for not thumb
        if y_pred[i] == 0 and target[i] == 0:
            matrix[0][0] += 1
        elif y_pred[i] == 0 and target[i] == 1:
            matrix[0][1] += 1
        elif y_pred[i] == 1 and target[i] == 0:
            matrix[1][0] += 1
        elif y_pred[i] == 1 and target[i] == 1:
            matrix[1][1] += 1
    matrix = matrix/np.sum(matrix)
    return matrix

In [22]:
batch_size = 32
dropout = 0.2
n_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-6
input_shape = (-1, 1, ch, t)

torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

# train neural net
accuracies = []
for subj, features, labels in fileList:
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # Train the neural network

        model.train()
        data = X_res
        targets = y_res
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        for epoch in range(n_epochs):
            # print("epoch = ",epoch)
            model.train()

            # Forward pass
            predictions = model(data)
            loss = criterion(predictions, targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        
        pred = predict(torch.as_tensor(X_train.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred*finger)
    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    
    # final NeuralNet
    model = NeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the neural network
    model.train()
    data = features
    targets = torch.tensor(y_train, dtype=torch.int64).to(device)

    for epoch in range(n_epochs):
        # print("epoch = ",epoch)
        model.train()

        # Forward pass
        predictions = model(data)
        loss = criterion(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), f'pickles/models/{subj}_final.pt')

    print(f'trained {subj} {finger}')
    new_features = []

    for finger in range(5):
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))

        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        print(f'accuracy {subj} finger {finger} = {evaluate(pred, y_res)} {binary_confusion(pred, y_res)})')
        new_features.append(pred*finger)


    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    
    model = NeuralNet().to(device)
    model.load_state_dict(torch.load(f'pickles/models/{subj}_final.pt'))
    acc = evaluate(predict(features), torch.tensor(y_test, dtype=torch.int64).to(device))
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)

print(f'avg accuracy = {np.mean(accuracies)}')

/tmp/ipykernel_73526/2044805763.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)


trained subj_A 4
accuracy = 0.7846715328467153
accuracy subj_A finger 0 = 0.7846715328467153 [[0.70620438 0.13686131]
 [0.07846715 0.07846715]])
accuracy = 0.7864963503649635
accuracy subj_A finger 1 = 0.7864963503649635 [[0.70985401 0.10218978]
 [0.11131387 0.07664234]])
accuracy = 0.7664233576642335
accuracy subj_A finger 2 = 0.7664233576642335 [[0.68978102 0.1350365 ]
 [0.09854015 0.07664234]])
accuracy = 0.7737226277372263
accuracy subj_A finger 3 = 0.7737226277372263 [[0.72080292 0.15875912]
 [0.06751825 0.05291971]])
accuracy = 0.8193430656934306
accuracy subj_A finger 4 = 0.8193430656934306 [[0.75547445 0.11861314]
 [0.0620438  0.06386861]])
accuracy = 0.36496350364963503
FINAL accuracy subj_A = 0.36496350364963503 ***************************************


/tmp/ipykernel_73526/2044805763.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
/tmp/ipykernel_73526/2044805763.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)


trained subj_B 4
accuracy = 0.7753496503496503
accuracy subj_B finger 0 = 0.7753496503496503 [[0.66083916 0.09527972]
 [0.12937063 0.11451049]])
accuracy = 0.7884615384615384
accuracy subj_B finger 1 = 0.7884615384615384 [[0.75699301 0.13986014]
 [0.07167832 0.03146853]])
accuracy = 0.7229020979020979
accuracy subj_B finger 2 = 0.7229020979020979 [[0.66433566 0.12937063]
 [0.14772727 0.05856643]])
accuracy = 0.7709790209790209
accuracy subj_B finger 3 = 0.7709790209790209 [[0.66870629 0.11363636]
 [0.11538462 0.10227273]])
accuracy = 0.8155594405594405
accuracy subj_B finger 4 = 0.8155594405594405 [[0.6993007  0.09877622]
 [0.08566434 0.11625874]])


/tmp/ipykernel_73526/2044805763.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)


...

/tmp/ipykernel_73526/2044805763.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)


trained subj_H 4
accuracy = 0.7003610108303249
accuracy subj_H finger 0 = 0.7003610108303249 [[0.60649819 0.10830325]
 [0.19133574 0.09386282]])
accuracy = 0.7436823104693141
accuracy subj_H finger 1 = 0.7436823104693141 [[0.64981949 0.07942238]
 [0.17689531 0.09386282]])
accuracy = 0.7220216606498195
accuracy subj_H finger 2 = 0.7220216606498195 [[0.66064982 0.15162455]
 [0.12635379 0.06137184]])
accuracy = 0.7436823104693141
accuracy subj_H finger 3 = 0.7436823104693141 [[0.65703971 0.12635379]
 [0.1299639  0.0866426 ]])
accuracy = 0.7545126353790613
accuracy subj_H finger 4 = 0.7545126353790613 [[0.64259928 0.0866426 ]
 [0.15884477 0.11191336]])
accuracy = 0.36101083032490977
FINAL accuracy subj_H = 0.36101083032490977 ***************************************


/tmp/ipykernel_73526/2044805763.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
/tmp/ipykernel_73526/2044805763.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)


trained subj_I 4
accuracy = 0.8489583333333334
accuracy subj_I finger 0 = 0.8489583333333334 [[0.68923611 0.05381944]
 [0.09722222 0.15972222]])
accuracy = 0.7326388888888888
accuracy subj_I finger 1 = 0.7326388888888888 [[0.65798611 0.10763889]
 [0.15972222 0.07465278]])
accuracy = 0.7673611111111112
accuracy subj_I finger 2 = 0.7673611111111112 [[0.69791667 0.13541667]
 [0.09722222 0.06944444]])
accuracy = 0.7274305555555556
accuracy subj_I finger 3 = 0.7274305555555556 [[0.60243056 0.09548611]
 [0.17708333 0.125     ]])
accuracy = 0.7864583333333334
accuracy subj_I finger 4 = 0.7864583333333334 [[0.68402778 0.07638889]
 [0.13715278 0.10243056]])
accuracy = 0.3923611111111111
FINAL accuracy subj_I = 0.3923611111111111 ***************************************
avg accuracy = 0.38894087742906935


/tmp/ipykernel_73526/2044805763.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)


In [21]:
# ensemble w/ cnn + svm :eyes:
from sklearn import svm
from sklearn import metrics


batch_size = 32
dropout = 0.2
n_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-6
input_shape = (-1, 1, ch, t)

torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

# train neural net
accuracies = []
for subj, features, labels in fileList:
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # Train the neural network

        model.train()
        data = X_res
        targets = y_res
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        for epoch in range(n_epochs):
            # print("epoch = ",epoch)
            model.train()

            # Forward pass
            predictions = model(data)
            loss = criterion(predictions, targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        
        pred = predict(torch.as_tensor(X_train.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred.to('cpu')*finger)
    features = np.array(new_features).transpose()

    # final NeuralNet
    classifier = svm.SVC(kernel='rbf')
    classifier.fit(features, y_train)

    print(f'trained {subj} {finger}')
    new_features = []

    for finger in range(5):
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))

        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        print(f'accuracy {subj} finger {finger} = {evaluate(pred, y_res)} {binary_confusion(pred, y_res)})')
        new_features.append(pred.to('cpu')*finger)

    features = np.array(new_features).transpose()

    pred = classifier.predict(features)
    acc = metrics.accuracy_score(y_test, pred)
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)

print(f'avg accuracy = {np.mean(accuracies)}')

trained subj_A 4
accuracy = 0.7828467153284672
accuracy subj_A finger 0 = 0.7828467153284672 [[0.69890511 0.12043796]
 [0.09671533 0.08394161]])
accuracy = 0.791970802919708
accuracy subj_A finger 1 = 0.791970802919708 [[0.75       0.16240876]
 [0.04562044 0.0419708 ]])
accuracy = 0.7846715328467153
accuracy subj_A finger 2 = 0.7846715328467153 [[0.72262774 0.13138686]
 [0.08394161 0.0620438 ]])
accuracy = 0.7737226277372263
accuracy subj_A finger 3 = 0.7737226277372263 [[0.71350365 0.15145985]
 [0.07481752 0.06021898]])
accuracy = 0.7956204379562044
accuracy subj_A finger 4 = 0.7956204379562044 [[0.73722628 0.12773723]
 [0.07664234 0.05839416]])
FINAL accuracy subj_A = 0.3467153284671533 ***************************************
trained subj_B 4
accuracy = 0.791958041958042
accuracy subj_B finger 0 = 0.791958041958042 [[0.69318182 0.13636364]
 [0.07167832 0.09877622]])
accuracy = 0.8085664335664335
accuracy subj_B finger 1 = 0.8085664335664335 [[0.80856643 0.19143357]
 [0.         0.   